# Modeling Porous Bone Structures From Vibrational Data
This is the third and final example in this tutorial for data-driven reduced-order modeling in the frequency domain. Here, we will use the AAA algorithm to learn a reduced-order model from frequency domain data. While it is not necessary to complete the previous two tutorial examples to understand the method in this one, it is recommended to understand the differences of the AAA algorithm to the Loewner framework.

In this example, we have given frequency domain data for the vibrational response of a porous bone. This data has our usual duplet form

$$
(\mathfrak{j} \omega_{1}, g_{1}), \quad (\mathfrak{j} \omega_{2}, g_{2}), \quad \ldots, \quad (\mathfrak{j} \omega_{N}, g_{N}),
$$

where $\omega_{i} \in \mathbb{R}$ are the real frequencies and $g_{i} = G(\mathfrak{j} \omega_{i})$ are evaluations of the unknown transfer functions on the imaginary axis; $\mathfrak{j} = \sqrt{-1}$ is the imaginary unit. Our goal is to find a low-dimensional first-order model of the form

$$ \tag{6.1}
\widehat{G} = \widehat{\mathbf{c}}^{\mathsf{T}} (s \widehat{\mathbf{E}} - \widehat{\mathbf{A}})^{-1} \widehat{\mathbf{b}},
$$

with the matrices $\widehat{\mathbf{E}}, \widehat{\mathbf{A}} \in \mathbb{C}^{r \times r}$, $\widehat{\mathbf{c}}, \widehat{\mathbf{b}} \in \mathbb{C}^{r}$ and reasonably small order $r \ll N$ so that this new transfer function approximates the given data well

$$
  \widehat{G}(\mathfrak{j} \omega_{1}) \approx g_{1}, \quad \widehat{G}(\mathfrak{j} \omega_{2}) \approx g_{2}, \quad \ldots, \quad \widehat{G}(\mathfrak{j} \omega_{N}) \approx g_{N}.
$$

In the previous examples, we used the Loewner framework to create a reduced-order model for the given data with an interpolating transfer function. There are situations in which this may not the desired approach:
* In case of much data ($N$ large), interpolation leads to high-dimensional models.
* While empirically an approximate least-squares behavior has been observed for the Loewner framework when using rank truncations, there are no theoretical guarantees for this. Additionally, the error behavior of the Loewner framework when choosing different reduced orders $r$ is not predictable so that one typically computes differently sized approximations and verifies the errors with respect to the given data.
* When parts of the data are noisy, interpolation of the whole data is typically undesired.

Therefore, we consider a different data-driven modeling method for this example: the Adaptive Antoulas-Anderson (AAA) algorithm. In this method, the reduced-order model is constructed iteratively where in every step only the data corresponding to the worst case approximation errors is interpolated while the rest is fitted by solving a linear least-squares problem. The key to the AAA algorithm is the reformulation of the rational transfer function $(6.1)$ into the interpolating barycentric form

$$ \tag{6.2}
\widehat{G}(s) = \frac{\sum\limits_{i = 1}^{r} \frac{w_{i} h_{i}}{s - \lambda_{i}}}{1 + \sum\limits_{i = 1}^{r} \frac{w_{i}}{s - \lambda_{i}}},
$$

with the barycentric weights $w_{i} \in \mathbb{C}$, the transfer function values $h_{i} \in  \mathbb{C}$ and the expansion points $\lambda_{i} \in \mathbb{C}$.
By construction, the barycentric form satisfies the following interpolation conditions

$$
\widehat{G}(\lambda_{i}) = h_{i} \quad\text{if}~w_{i} \neq 0.
$$

This means that after selecting suitable interpolation points, we have the freedom of choosing the weights $w_{i}$ to improve the approximation quality of the transfer function. In AAA, the weights $w_{i}$ are selected as the solution to a linearized least-sqares problem.

With the basic idea explained, let's dive into the tutorial example. Further details about the AAA will be explained on the way.

First, we load our usual Python packages for computations and visualizations.

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.sparse as sparse
import matplotlib.pyplot as plt

As in the previous examples, we include a utilities file [utils.py](./utils.py), which implements some fundamental routines for easy visualizations.

In [ ]:
import utils

utils.configure_matplotlib(latex_is_installed = True)

This file also contains the definition of the class `BarycentricModel`, which implements the barycentric realization $(6.2)$ of the transfer function of first-order systems using the parameters `weights` for the barycentric weights, `tfvalues` for the transfer function values and `tfpoints` for the barycentric expansion points. Important methods of this class are:
* `toStateSpace`, which returns the matrices `A`, `b` and `c` for the classical state space realization $(6.1)$ following the formulas below. Note that no `E` matrix is returned because $\widehat{\mathbf{E}} = \mathbf{I}_{r}$ holds. For the rest we have
$$
\widehat{\mathbf{A}} = \begin{bmatrix} \lambda_{1} & & \\ & \ddots & \\ & & \lambda_{r} \end{bmatrix} - \widehat{\mathbf{b}} \mathbf{1}_{r}^{\mathsf{T}}, \quad
\widehat{\mathbf{b}} = \begin{bmatrix} w_{1} \\ \vdots \\ w_{r} \end{bmatrix}, \quad
\widehat{\mathbf{c}} = \begin{bmatrix} h_{1} \\ \vdots \\ h_{r} \end{bmatrix}.
$$

* `transfer_function`, which takes an array of complex points as inputs and returns the frequency response $\widehat{G}(s)$ of the current barycentric model at these points.

# Step 1: Loading the data
Let's begin by loading the given data set. This data set contains the real-valued frequencies $\omega_{1}, \ldots, \omega_{N}$ and the frequency responses so that

$$
g_{1} = G(\mathfrak{j} \omega_{1}), \quad \ldots, \quad g_{N} = G(\mathfrak{j} \omega_{N}),
$$

where $G(s)$ is an unknown transfer function.

In [ ]:
# Load the given frequency data.
frequencies, responses = utils.load_data()

# Visualize given data.
ax = utils.plot_response(
    frequencies,
    responses,
    linestyle = "",
    marker    = ".",
    label     = "Transfer function data"
)
ax.legend()
plt.show()

The visualization shows that the given data has a lot of peaks and sinks. In general, we could define a subset of these data to be trusted for the interpolation step in AAA. However, this is not necessary for this given data set and we consider all given points for potential interpolation.

Before moving on, we have to take care of two things. First, we will modify the given data during the iteration and therefore, save the data points in new variables. Second, we want to define some importance weights for the data. This allows us to easily change the approximation objective as needed. Typical choices for weighting coefficients $\eta_{i}$ are
* $\eta_{i} = 1$ for $i = 1, \ldots, N$, which is equivalent to considering absolute errors in interpolation and least-squares, or
* $\eta_{i} = \frac{1}{\lvert g_{i} \rvert}$ for $i = 1, \ldots, N$; which is equivalent to using relative errors in interpolation and least-squares.

For now we are going to use $\eta_{i} = 1$.

In [ ]:
training_frequencies = 1j * frequencies
training_responses   = responses
training_weights     = np.ones((len(training_responses),))

# Step 2: Initialization and first interpolation point
If no prior model is given, the AAA algorithm starts from zero, i.e.,

$$
\widehat{G} \equiv 0.
$$

For this, we initialize the barycentric model without any weights or values.

In [ ]:
aaa_model = utils.BarycentricModel()

print(
    "Content of the barycentric parameters",
    f"weights : {aaa_model.weights}",
    f"tfvalues: {aaa_model.tfvalues}",
    f"tfpoints: {aaa_model.tfpoints}",
    sep = "\n"
)

In the beginning of every AAA iteriaton, we need to choose a point from the current data set for interpolation. This is typically done by computing the frequency where the maximum weighted error between the training data set and the current reduced-order transfer function occurs.

In [ ]:
def choose_interpolation_point(model, frequencies, responses, dataweights):
    """Find the maximizer of the approximation error."""
    model_responses   = model.transfer_function(frequencies)
    error_responses = dataweights * np.abs(model_responses - responses)

    index = error_responses.argmax()
    error = error_responses[index]
    
    return (index, error)

# Find error maximizer.
index, error = choose_interpolation_point(
    aaa_model,
    training_frequencies,
    training_responses,
    training_weights
)

# Visualization of the chosen point.
ax = utils.plot_response(
    frequencies,
    responses,
    linestyle = "",
    marker    = ".",
    label     = "Transfer function data"
)
ax = utils.plot_response(
    training_frequencies[index].imag,
    training_responses[index],
    ax        = ax,
    linestyle = "",
    color     = "k",
    marker    = "x",
    label     = "Expansion point"
)
ax.legend()
plt.show()

print(
    "Approximation error of zero model:", error
)

The function has given us the index of the transfer function value of largest magnitude because this value has the largest distance from $0$ and is therefore the maximizer. Additionally, we compute the current discrete error value our approximation is making. This will play an important role for stopping the adaptive AAA algorithm.

Once the interpolation point is chosen, we need to remove the corresponding data from the data sets and add the point to the barycentric form.

In [ ]:
# Extend model.
aaa_model.tfvalues = np.array([training_responses[index]])
aaa_model.tfpoints = np.array([training_frequencies[index]])

# Remove data points.
training_responses   = np.delete(training_responses, index)
training_frequencies = np.delete(training_frequencies, index)
training_weights     = np.delete(training_weights, index)

# Verify removal.
print(
    "Sizes of new data sets:",
    training_responses.shape,
    training_frequencies.shape,
    training_weights.shape,
    sep = "\n"
)

# Step 2: Least-squares fit
At this point, we only have the transfer function value and expansion point in the barycentric form. For the formulation of the transfer function in teh barycenctric form $(6.2)$, we still need the corresponding weight $w_{1}$. To choose this weight, we are going to fit the remaining data in a least-squares sense. In general, the minimization roblem

$$
\min_{w_{i}} \sum\limits_{i = 1}^{N - r} \eta_{i} \lvert \widehat{G}(\mathfrak{j}\omega_{i}) - g_{i} \rvert^{2},
$$

with $r = 1$ for now, is a rational (nonlinear) least-squares problem. Even finding local minimizers is a difficult and computationally heavy task. Therefore, the AAA algorithm uses that the transfer function is a rational function

$$
\widehat{G}(s) = \frac{n(s)}{d(s)},
$$

for numerator and denominator $n(s)$ and $d(s)$, respectively. With these, we consider the linearized least-squares problem

$$
\min_{w_{i}} \sum\limits_{i = 1}^{N - r} \eta_{i}^{2} \lvert n(\mathfrak{j}\omega_{i}) - d(\mathfrak{j}\omega_{i}) g_{i} \rvert^{2}.
$$

This problem is easy to solve for a global optimum. For numerical stability, the numerator and denominator are chosen from the barycentric form $(6.2)$ so that we have

$$
n(s) = \sum\limits_{i = 1}^{r}\frac{w_{i} h_{i}}{s - \lambda_{i}} \quad\text{and}\quad d(s) = 1 + \sum\limits_{i = 1}^{r}\frac{w_{i}}{s - \lambda_{i}},
$$

Formulating this linear least-squares problem in matrix form, we quickly see that we have to solve

$$
-\boldsymbol{\eta} \mathbb{L} \mathbf{w} = \boldsymbol{\eta} \mathbf{g},
$$

where $\boldsymbol{\eta} = \operatorname{diag}(\eta_{1}, \ldots, \eta_{N - r})$ is the matrix of all weighting coefficients, $\mathbf{w}$ is the vector of all unknown barycentric weights, $\mathbf{g}$ is the vector of the remaining transfer function response data, and $\mathbb{L}$ is a Loewner matrix of the form

$$
  \mathbb{L}_{i, j} = \frac{g_{i} - h_{j}}{\mathfrak{j}\omega_{i} - \lambda_{j}}.
$$

We can solve this linear least-squares problem with any standard least-squares solver.

In [ ]:
def least_squares_weights(model, frequencies, responses, dataweights):
    """Solve linear least-squares problem to find barycentric weights."""

    # Set up Loewner matrix.
    L = (responses.reshape(len(responses), 1) - model.tfvalues) \
        / (frequencies.reshape(len(frequencies), 1) - model.tfpoints)

    # Solve linear least-squares problem.
    lsq_results = la.lstsq(
        sparse.diags(dataweights) @ L,
        -dataweights * responses
    )

    # Return only least-squares solution.
    return lsq_results[0]

# Compute least-squares weight.
aaa_model.weights = least_squares_weights(
    aaa_model,
    training_frequencies,
    training_responses,
    training_weights
)

With the weights computed, we have now a complete reduced-order model of dimension $r = 1$.

In [ ]:
Ar, br, cr = aaa_model.toStateSpace()

print(
    "The system matrices are",
    f"Ar = {Ar}",
    f"br = {br}",
    f"cr = {cr}",
    sep = "\n"
)

Let's check that everything went correctly. First, we satisfy the interpolation condition in the selected point.

In [ ]:
print(
    "Data response:", responses[index],
    "ROM response:", aaa_model.transfer_function([1j * frequencies[index]])[0]
)

Also, we can have a look at how well we are approximating the overall data set at this point.

In [ ]:
model_responses = aaa_model.transfer_function(1j * frequencies)

ax = utils.plot_comparison(
    frequencies,
    responses,
    model_responses,
    np.abs(responses - model_responses),
    aaa_model.tfpoints,
    aaa_model.tfvalues
)
ax[0].legend()
plt.show()

We can clearly identify in the relative error the frequency at which we are interpolating. Looking at the general transfer function behavior, we can believe that the rest of the data is matched in an absolute (linearized) least-squares sense.

# Step 3: Assembling the iteration
The two steps from above, namely the choice of an interpolation point and the least-squares fitting of the non-interpolated data, together are one iteration of the AAA algorithm. Repeating these iterations will construct an accurate model. This is what we are doing next.

In [ ]:
def aaa(
    frequencies,
    responses,
    dataweights = None,
    maxiter = 100,
    tolerance = 1e-6
):
    """Basic implementation of the AAA algorithm."""
    # Initialization.
    aaa_model     = utils.BarycentricModel()
    error_history = np.zeros((maxiter,))

    # Default data weights are 1.
    if dataweights is None:
        dataweights = np.ones((len(frequencies,)))

    for iter in range(0, maxiter):
        # Find error maximizing frequency and current approximation error
        index, error = choose_interpolation_point(
            aaa_model,
            frequencies,
            responses,
            dataweights
        )

        error_history[iter] = error

        # Stopping criterion.
        if error < tolerance:
            return aaa_model, error_history[0:iter+1]
        
        # Extend model.
        if iter == 0:
            aaa_model.tfvalues = np.array([responses[index]])
            aaa_model.tfpoints = np.array([frequencies[index]])
        else:
            aaa_model.tfvalues = np.append(aaa_model.tfvalues, responses[index])
            aaa_model.tfpoints = np.append(aaa_model.tfpoints, frequencies[index])

        # Remove interpolation data.
        responses   = np.delete(responses, index)
        frequencies = np.delete(frequencies, index)
        dataweights = np.delete(dataweights, index)

        # Compute the barycentric weights.
        aaa_model.weights = least_squares_weights(
            aaa_model,
            frequencies,
            responses,
            dataweights
        )

    return aaa_model, error_history
        

With our AAA algorithm ready to go, we can experiment using our given data to construct suitable approximations. The following code computes a AAA model with maximum order $r = 10$ or up to the error tolerance $10^{-4}$. Try to experiment a bit with different accuracies and maximum model sizes. Can you find a model size at which the error tolerance $10^{-4}$ is reached?

In [ ]:
aaa_model, error_history = aaa(
    1j * frequencies,
    responses,
    maxiter = 10,
    tolerance = 1e-4
)

# Determine size of approximation.
print(
    "The reduced-order model has the dimension r =",
    len(aaa_model.weights)
)

# Visualize results.
model_responses = aaa_model.transfer_function(1j * frequencies)

ax = utils.plot_comparison(
    frequencies,
    responses,
    model_responses,
    np.abs(responses - model_responses),
    aaa_model.tfpoints,
    aaa_model.tfvalues
)
ax[0].legend()
plt.show()

Additionally to the relative response error, we can also have a look at the weighted error that we computed during the iteration.

In [ ]:
utils.plot_error_history(error_history)
plt.show()

**Important:** The weighted error in the AAA iteration will rarely decay monotonically. The reason is that instead of the nonlinear least-squares problem that involves the actual transfer function, we only solve the linearized version with the numerator and denominator from the barycentric form. That means that the least-squares error has only a weak relation to the true weighted error we are trying to minimize.

# Step 4: Using relative data weights
So far, all our experiments have run with the data weights being $1$ (equivalent to not weighting the data at all). As mentioned earlier, this corresponds to fitting the data with the barycentric form to minimize the absolute approximation error. There are infinitely many other choices for how weight the data. A practically important one is to include the variance of the data to allocate higher importance to trustworthy data over noisy data. However, this needs insight knowledge into the data generation process.

A different idea for weighting is based on a favored measure to determine the accuracy of approximations: the *pointwise relative error*. For this approach, we set up our weights to be

$$
\eta_{i} = \frac{1}{\lvert g_{i} \rvert} \quad \text{for}~i = 1, \ldots, N.
$$

In [ ]:
dataweights = 1 / np.abs(responses)

Now we can use our AAA algorithm again, to fit the given data with this new type of weighting.

**Important:** Before you run the algorithm and play around with the maximum orders and tolerances, note that the interpretation of our stopping criterion has changed. While we are still aiming to minimize the weighted error, the interpretation of the weighted error has changed from the absolute approximation error (no weighting) to the relative approximation error. Therefore, it will be significantly harder to reach small tolerances before becoming too large with our approximation order.

In [ ]:
aaa_model_rel, error_history_rel = aaa(
    1j * frequencies,
    responses,
    dataweights = dataweights,
    maxiter = 1,
    tolerance = 1e-2
)

# Determine size of approximation.
print(
    "The reduced-order model has the dimension r =",
    len(aaa_model_rel.weights)
)

# Visualize results.
model_responses_rel = aaa_model_rel.transfer_function(1j * frequencies)

ax = utils.plot_comparison(
    frequencies,
    responses,
    model_responses_rel,
    dataweights * np.abs(responses - model_responses_rel),
    aaa_model_rel.tfpoints,
    aaa_model_rel.tfvalues
)
ax[0].legend()
plt.show()

We can clearly see that with the relative data weights, the approximation tends towards fitting the frequency response data of smallest magnitude first. This makes sense since small magnitudes lead to large relative approximation errors.

If you run the AAA algorithm with the relative weighting above for `maxiter = 1`, you will see some very peculiar behavior: the algorithm will always select the first response data point from the given list. This comes from the fact that we begin the AAA algorithm with the zero model ($\widehat{G} \equiv 0$). Therefore, the maximization of the weighted error in the first step of the algorithm becomes

$$
\arg\max_{i} \eta_{i} \lvert g_{i} - \widehat{G}(\mathfrak{j} \omega_{i}) \rvert =  \arg\max_{i} \frac{1}{\lvert g_{i} \rvert} \lvert g_{i} \rvert = \arg\max_{i} 1.
$$

In other words, we have the exact same weighted approximation error for the complete response data set and the algorithm simply takes the first point of these. This behavior is a disadvantage only occurring in the first step of AAA. As a remedy, it is typically proposed to ignore the relative weighting only in the first step of the method and use instead the absolute weighting. The corresponding appropriate code modification would go beyond the scope of this tutorial and is left as homework to the user.